# 04 Advance Voting Stations Analysis**Question**: How much coverage do the 400 advance voting station locations provide?**Purpose**: Analyze advance voting station data from heypoom's geocoded dataset to understand coverage relative to the complete 95k ballot unit dataset.**Tech Summary**:- **Input**: `../data/poom_ballot_location.csv` (444 geocoded locations from Google Places API), `../data/SHP ECT attributes/2566_TH_ECT_attributes.shp` (400 electoral districts)- **Process**:  - Load ECT electoral district boundaries (400 constituencies)  - Load ballot locations from heypoom's geocoded CSV    - Perform spatial join to map locations to electoral districts  - Visualize coverage with KeplerGL- **Output**: `data/regions_clean.csv`, KeplerGL visualization- **Dependencies**: geopandas, pandas, keplergl**Status**: ✓ Stable**Key Findings**:- Contains ~444 geocoded locations (only **0.47%** of total 95,249 voting units)- Does not significantly contribute to the main 90k location dataset- Primarily useful as reference/validation data- Data source: [heypoom/voting-station-locations](https://github.com/heypoom/voting-station-locations)---

In [1]:
import pandas as pd
import geopandas as gpd

regions = gpd.read_file('../data/SHP ECT attributes/2566_TH_ECT_attributes.shp')
ballots = pd.read_csv('data/poom_ballot_location.csv')
ballots = gpd.GeoDataFrame(ballots, geometry=gpd.points_from_xy(ballots.longitude, ballots.latitude), crs='EPSG:4326')

In [2]:
regions.head()

,P_name,CONS_no,ID_name,geometry
0,กระบี่,3,กระบี่_3,"MULTIPOLYGON (((99.2622 7.66229, 99.26186 7.66..."
1,กระบี่,1,กระบี่_1,"MULTIPOLYGON (((99.07948 7.89818, 99.07964 7.8..."
2,กระบี่,2,กระบี่_2,"MULTIPOLYGON (((98.72608 8.24924, 98.72538 8.2..."
3,กรุงเทพมหานคร,29,กรุงเทพมหานคร_29,"POLYGON ((100.33093 13.70413, 100.33179 13.704..."
4,กรุงเทพมหานคร,28,กรุงเทพมหานคร_28,"POLYGON ((100.40482 13.66259, 100.40481 13.662..."


In [3]:
ballots.head()

,title,province,order,google_title,address,place_id,status,latitude,longitude,geometry
0,โรงเรียนอนุบาลกระบี่ ถนนท่าเรือ อำเภอเมืองกระบี่,กระบี่,1,โรงเรียนอนุบาลกระบี่,"1 Utarakit Rd, Pak Nam, Mueang Krabi District,...",ChIJ44eKd7mUUTARnz_JuJNdjUo,OPERATIONAL,8.050567,98.908363,POINT (98.90836 8.05057)
1,โรงเรียนอ่าวลึกประชาสรรค์ อำเภออ่าวลึก,กระบี่,2,Ao Luek Prachasan School,"9PXV+X3C, Ao Luek Tai, Ao Luek District, Krabi...",ChIJQ5EYNfcJUTAROAKpO1NcDQg,OPERATIONAL,8.399940,98.742651,POINT (98.74265 8.39994)
2,หอประชุมโรงเรียนคลองท่อมราษฎร์รังสรรค์ อำเภอคล...,กระบี่,3,Khlong Thom Ratrangsan School,"207 Petchkasem Rd, Khlong Thom Tai, Khlong Tho...",ChIJV-bOFCv0UTARUgh9EoEBmAU,OPERATIONAL,7.947897,99.145669,POINT (99.14567 7.9479)
3,บริเวณลานจอดรถสำนักงานเขตพระนคร,กรุงเทพ,1,ลานจอดรถ J Park เสาชิงช้า,"83 Bamrung Mueang Rd, Sao Chingcha, Phra Nakho...",ChIJHQaeVXSZ4jARaW8LwusBaro,OPERATIONAL,13.752082,100.498755,POINT (100.49875 13.75208)
4,บริเวณสำนักงานเขตป้อมปราบศัตรูพ่าย,กรุงเทพ,1,Pom Prap Sattru Phai District Office,"50 Thanon Suppha Mit, Wat Sommanat, Pom Prap S...",ChIJceCxnj-Z4jARzcJ6U54enY0,OPERATIONAL,13.758240,100.513072,POINT (100.51307 13.75824)


# ตอนนี้เรามีข้อมูลกี่หน่วย และคิดเป็นกี่เปอร์เซ็นต์

In [4]:
total_units = 95249 
disconvered = ballots.count().title
disconvered, disconvered / total_units * 100

(np.int64(444), np.float64(0.4661466262112988))

In [5]:
regions.shape

(400, 4)

# Aggregate sumation under 1 ballot region

In [6]:
regions.sjoin(ballots).groupby("index_right").size().rename("points")

index_right
0      1
1      1
2      1
3      1
4      1
      ..
439    1
440    1
441    1
442    1
443    1
Name: points, Length: 442, dtype: int64

In [7]:
regions = regions.join(
    gpd.sjoin(ballots, regions).groupby("index_right").size().rename("N_Ballots"),
    how="left",
)
regions.head()

,P_name,CONS_no,ID_name,geometry,N_Ballots
0,กระบี่,3,กระบี่_3,"MULTIPOLYGON (((99.2622 7.66229, 99.26186 7.66...",1.0
1,กระบี่,1,กระบี่_1,"MULTIPOLYGON (((99.07948 7.89818, 99.07964 7.8...",1.0
2,กระบี่,2,กระบี่_2,"MULTIPOLYGON (((98.72608 8.24924, 98.72538 8.2...",1.0
3,กรุงเทพมหานคร,29,กรุงเทพมหานคร_29,"POLYGON ((100.33093 13.70413, 100.33179 13.704...",2.0
4,กรุงเทพมหานคร,28,กรุงเทพมหานคร_28,"POLYGON ((100.40482 13.66259, 100.40481 13.662...",NaN


In [8]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(regions):
    # Replace missing values with 0 in column: 'N_Ballots'
    regions = regions.fillna({'N_Ballots': 0})
    regions_clean = regions.astype({'N_Ballots': 'uint16'})

    return regions

regions_clean = clean_data(regions.copy())
regions_clean.head()

,P_name,CONS_no,ID_name,geometry,N_Ballots
0,กระบี่,3,กระบี่_3,"MULTIPOLYGON (((99.2622 7.66229, 99.26186 7.66...",1.0
1,กระบี่,1,กระบี่_1,"MULTIPOLYGON (((99.07948 7.89818, 99.07964 7.8...",1.0
2,กระบี่,2,กระบี่_2,"MULTIPOLYGON (((98.72608 8.24924, 98.72538 8.2...",1.0
3,กรุงเทพมหานคร,29,กรุงเทพมหานคร_29,"POLYGON ((100.33093 13.70413, 100.33179 13.704...",2.0
4,กรุงเทพมหานคร,28,กรุงเทพมหานคร_28,"POLYGON ((100.40482 13.66259, 100.40481 13.662...",0.0


In [9]:
from keplergl import KeplerGl


/home/ben/ddd/ninyawee/ballot-location/.venv/lib/python3.12/site-packages/keplergl/keplergl.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


In [10]:

map_1 = KeplerGl(height=600)
# map_1.height = 600
map_1.add_data(data=regions_clean, name='เขตเลือกตั้ง 2566')
# map_1.add_data(data=ballots, name='สถานที่ลงคะแนนเลือกตั้ง Poom')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'เขตเลือกตั้ง 2566': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…

In [25]:
regions_clean.to_csv('data/regions_clean.csv', index=False)